###Python setup

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 9.0MB/s 
     |████████████████████████████████| 194kB 14.5MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 1.2MB 19.2MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 12.8MB 37kB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
from fastbook import *
from fastai.text.all import *
import pandas as pd

###Load abstract data

In [5]:
### Load data from github
url = 'https://github.com/andreasebbehoj/EASD-abstract-nlp/raw/main/2_Data/Abstracts.xlsx'
df = pd.read_excel(url)
df.head(n=1)

,id,year,page,oral,title,abstract,authornames,authoraddresses,correspondenceaddress,authorkeywords,emtreedrugindextermsmajorfocus,emtreedrugindexterms,emtreemedicalindextermsmajorfocu,emtreemedicalindexterms,casregistrynumbers,embaselink
0,1,2009,7,NaN,One-year exenatide treatment improves beta cell response in metformin treated patients with type 2 diabetes which is sustained after 5 weeks discontinuation of treatment,"BACKGROUND AND AIMS: Recently we reported exenatide (EX) improved hy-perglycaemic clamp derived measures of Î²-cell function, compared to insulin glargine (IG) in metformin (MET) treated patients with type 2 diabetes. Here we report the effect of EX treatment on the Î²-cell response to a more physi-ological stimulus, i.e. a mixed meal. MATERIALS AND METHODS: In this investigator-designed study, we compared the effects of both EX and IG treatment on prandial Î²-cell function during an 8 hour, high fat, double mixed meal test. Sixty-nine MET treated patients with T2D (pre-treatment (PT) age ...","Bunck M.C., Mari A., Comer A., Eliasson B., Shaginian R.M., Wu Y., Yan P., Heine R.J., Smith U., Taskinen M.-R., Yki-Jarvinen H., Diamant M.","(Bunck M.C.; Heine R.J.; Diamant M.) Department of Endocrinology, Diabetes Center, VU University Medical Center, Amsterdam, Netherlands. , (Mari A.) Institute of Biomedical Engineering, Padua, Italy. , (Comer A.; Taskinen M.-R.; Yki-Jarvinen H.) Department of Medicine, Helsinki University Central Hospital, Finland. , (Eliasson B.; Smith U.) Lundberg Laboratory for Diabetes Research, Sahlgrenska University Hospital, Goteborg, Sweden. , (Shaginian R.M.) Eli Lilly, Houten, Netherlands. , (Wu Y.; Yan P.) Amylin Pharmaceuticals, Inc., San Diego, United States. , (Heine R.J.) Eli Lilly, Indianap...","M.C. Bunck, Department of Endocrinology, Diabetes Center, VU University Medical Center, Amsterdam, Netherlands.",NaN,"exendin 4, metformin","glucose, immunoglobulin, insulin glargine, peptide","diabetes mellitus, patient","cell function, clamp, dose response, insulin release, mathematical model, meal, secretion (process), stimulus, therapy",NaN,https://www.embase.com/search/results?subaction=viewrecord&id=L70067493&from=export


In [6]:
### Load data into language model datablock
dls_lm = DataBlock(
    blocks=TextBlock.from_df('abstract', is_lm=True), ## abstract is the var in df that contains text
    get_x=ColReader('text'), ## should always be 'text' irrespective of var name (according to FastAI tutorial)
    splitter=RandomSplitter(0.2, seed=42) ## Split into train/validation data
).dataloaders(df, bs=128, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
### Show batch after automatic text-preprocessing  
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos xxup background xxup and xxup aims : xxmaj delayed - release metformin ( met xxup dr ) is being developed for t2dmpatients with chronic kidney disease ( ckd xxmaj stage 3b or 4 ) . xxmaj by utilizing an enteric coating to bypass the stomach and upper intestine , xxmaj met xxup dr delivers metformin to the lower bowel to retain its gut - based glucose - lowering properties but with greatly reduced absorption and systemic exposure . xxmaj","xxup background xxup and xxup aims : xxmaj delayed - release metformin ( met xxup dr ) is being developed for t2dmpatients with chronic kidney disease ( ckd xxmaj stage 3b or 4 ) . xxmaj by utilizing an enteric coating to bypass the stomach and upper intestine , xxmaj met xxup dr delivers metformin to the lower bowel to retain its gut - based glucose - lowering properties but with greatly reduced absorption and systemic exposure . xxmaj this"
1,"a defect in incretin secretion is impaired in women with xxup gdm remains to be ascertained . xxmaj therefore , we have determined xxup glp-1 secretion in response to oral glucose load in pregnant women with normal glucose tolerance ( ngt ) and with xxup gdm . xxup materials xxup and xxup methods : xxunk xxup ogtt ( evaluated according to xxmaj carpenter and xxmaj coustan criteria ) was performed in 12 xxup gdm and 16 xxup ngt women at","defect in incretin secretion is impaired in women with xxup gdm remains to be ascertained . xxmaj therefore , we have determined xxup glp-1 secretion in response to oral glucose load in pregnant women with normal glucose tolerance ( ngt ) and with xxup gdm . xxup materials xxup and xxup methods : xxunk xxup ogtt ( evaluated according to xxmaj carpenter and xxmaj coustan criteria ) was performed in 12 xxup gdm and 16 xxup ngt women at xxunk"
2,"xxup background xxup and xxup aims : xxmaj agents that act on the renin - angiotensin - aldosterone system ( raas ) are considered drugs of choice for the treatment of hypertension in patients with diabetes mellitus . xxmaj direct renin inhibitors ( xxunk ) bind to and inhibit renin , which catalyzes the rate - limiting step of the xxup raas . xxmaj this 8-week , double - blind study compared the blood pressure lowering ( bp ) efficacy","background xxup and xxup aims : xxmaj agents that act on the renin - angiotensin - aldosterone system ( raas ) are considered drugs of choice for the treatment of hypertension in patients with diabetes mellitus . xxmaj direct renin inhibitors ( xxunk ) bind to and inhibit renin , which catalyzes the rate - limiting step of the xxup raas . xxmaj this 8-week , double - blind study compared the blood pressure lowering ( bp ) efficacy and"


##Train language model
First part loads the pre-trained model (based on English wiki text). Then the model gets fine-tuned on the EASD abstracts. 

This model's final layer can predict the next word in a sentence. This can be used for auto-generating (poor) abstracts. But the model can also be trained for classifying abstracts in those accepted for an oral presentation vs only a poster presentation.

In [9]:
### Load pretrained model
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [10]:
### Fine-tune last layer on our data
learn.fit_one_cycle(1, 2e-2) # Note that with 13k abstracts each epoch takes ~6 min

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.536837,3.423170,0.379316,30.666485,05:14


In [11]:
### Unfreeze and continued fine-tuning of all layers 
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3) # First try with 10 cycles lead to increased valid_loss after 6 cycles. Change to 5 cycles?

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.143466,3.261853,0.399243,26.097857,05:46
1,3.006297,3.158729,0.411886,23.540648,05:45
2,2.856818,3.080768,0.421600,21.775116,05:45
3,2.707183,3.039272,0.427671,20.890020,05:45
4,2.569106,3.024357,0.431028,20.580774,05:45
5,2.438265,3.028320,0.432231,20.662481,05:45
6,2.306128,3.046739,0.432804,21.046591,05:46
7,2.199559,3.071510,0.432493,21.574451,05:47
8,2.106103,3.089130,0.432053,21.957970,05:45
9,2.070040,3.099639,0.431548,22.189932,05:45


### Save models

In [18]:
### Save model
learn.save('lm_complete') # All layers
learn.save_encoder('lm_encode') # All except last layer (for classification model)

### Copy
!cp "/content/models/lm_complete.pth"   "/content/gdrive/MyDrive/Colab Notebooks/EASD-abstract-nlp/3_Models/lm_complete.pth"
!cp "/content/models/lm_encode.pth"     "/content/gdrive/MyDrive/Colab Notebooks/EASD-abstract-nlp/3_Models/lm_encode.pth"

### Use language model to auto-generate abstracts
Note: Don't submit them to EASD or anywhere else! The abstracts are absolutely awful and without any rhyme or reason. Also, you'll like break some sort of conference rule and get banned.

In [ ]:
### Load model
learn.load("/content/gdrive/MyDrive/Colab Notebooks/EASD-abstract-nlp/3_Models/lm_complete.pth")

In [25]:
### Abstract-generator
TEXTSTART = ["BACKGROUND AND AIM:", " MATERIALS AND METHODS:", " RESULTS:", " CONCLUSION:"]
N_WORDS = [80, 140, 180, 30]
N_SENTENCES = 4
Abstract = ""
text = ""

for n in range(N_SENTENCES):
  text =  Abstract + TEXTSTART[n]
  Abstract = learn.predict(text, N_WORDS[n], temperature=0.75)

print((Abstract))

BACKGROUND AND AIM : Human embryonic stem cells ( escs ) and their progenitors size are essential for regenerative medicine for cell - type dynamics and regeneration . We have recently observed that the expression of ECM proteins in the pancreas is enhanced in diabetes as has been suggested by the use of the protein 5'-utr . However , the molecular mechanism of this action remains unknown , particularly in beta cells . The present study is designed to MATERIALS AND METHODS : Live cell imaging was performed in alphatc1 cells ( mean number of cells collected from the pancreas of 15 , 28 , 60 , 90 , 90 and 96 cells ) cells . The proliferation and apoptosis were measured by TUNEL , Ki67 and Ki67 staining , respectively . Cell proliferation was measured after cell cycle induction by using the cell viability kit . The same cells were treated with 1 or 7 mm of Insulin for 24 hours or 1 mm , respectively . Cell viability was evaluated by caspase-3 cleavage , cell proliferation by brdu incorpor